In [1]:
import torch
from tensordict.tensordict import TensorDict

In [2]:
a = torch.rand(3, 4)
b = torch.rand(3, 4, 5)
tensordict = TensorDict({"a": a, "b": b}, batch_size=[3, 4])

indexed_tensordict = tensordict[:2, 1]
assert indexed_tensordict["a"].shape == torch.Size([2])
assert indexed_tensordict["b"].shape == torch.Size([2, 5])

In [4]:
tensordict[:1]

TensorDict(
    fields={
        a: Tensor(shape=torch.Size([1, 4]), device=cpu, dtype=torch.float32, is_shared=False),
        b: Tensor(shape=torch.Size([1, 4, 5]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([1, 4]),
    device=None,
    is_shared=False)

In [5]:
tensordict[:2]

TensorDict(
    fields={
        a: Tensor(shape=torch.Size([2, 4]), device=cpu, dtype=torch.float32, is_shared=False),
        b: Tensor(shape=torch.Size([2, 4, 5]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([2, 4]),
    device=None,
    is_shared=False)

In [6]:
tensordict[:, 0:2]

TensorDict(
    fields={
        a: Tensor(shape=torch.Size([3, 2]), device=cpu, dtype=torch.float32, is_shared=False),
        b: Tensor(shape=torch.Size([3, 2, 5]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([3, 2]),
    device=None,
    is_shared=False)

In [7]:
print(tensordict[:, :]['a'])
tensordict[:, :]['b'].shape


tensor([[0.8008, 0.1869, 0.8878, 0.8746],
        [0.8891, 0.7147, 0.2167, 0.9934],
        [0.5049, 0.3155, 0.7060, 0.9949]])


torch.Size([3, 4, 5])

In [8]:
import envpool
from modules.utils import set_seed, make_batched_env


/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [9]:
from omegaconf import OmegaConf

cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo_trainer.yaml"
nn_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/nn/nn.yaml"
ppo_cfg_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ppo/ppo.yaml"
ccnn_cfg_img_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ccnn_img/ccnn_img.yaml"
ccnn_cfg_seq_path = "/home/kukjin/kukjin/Projects/MultiEnvRL/DARL_transformer/configs/ccnn_seq/ccnn_seq.yaml"

cfg = OmegaConf.load(cfg_path)
nn_cfg = OmegaConf.load(nn_cfg_path)
ppo_cfg = OmegaConf.load(ppo_cfg_path)
ccnn_seq_cfg = OmegaConf.load(ccnn_cfg_img_path)
ccnn_img_cfg = OmegaConf.load(ccnn_cfg_seq_path)

cfg.nn = nn_cfg
cfg.ppo = ppo_cfg
cfg.ccnn_seq = ccnn_seq_cfg
cfg.ccnn_img = ccnn_img_cfg

cfg

{'defaults': [{'ppo': 'ppo'}, {'nn': 'nn'}, {'ccnn_img': 'ccnn_img'}, {'ccnn_seq': 'ccnn_seq'}, '_self_'], 'hydra': {'run': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}'}}, 'experiment': {'env_ids': ['CartPole-v1'], 'seed': 42, 'max_episode_steps': 1000, 'num_rollout_steps': 128, 'num_envs': 64, 'total_timesteps': 10000000, 'save_ckpt': False, 'num_checkpoints': 20, 'print_interval': 100, 'stop_after_epochs': 500, 'capture_video': False, 'device': 1, 'cuda': True, 'torch_deterministic': True, 'resume': False, 'resume_update_idx': 0, 'resume_dir': 'None'}, 'evaluation': {'eval_seed': 3142, 'every': 8, 'num_eval': 5, 'num_test_envs': 5}, 'wandb': {'mode': 'online', 'project': 'DomainAgnosticRL', 'entity': None, 'name': None, 'group': None, 'tags': None, 'notes': None}, 'paths': {'dir': 'outputs/${now:%Y-%m-%d/%H-%M-%S}', 'log': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/runs', 'video': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/videos', 'checkpoints': 'outputs/${now:%Y-%m-%d/%H-%M-%S}/checkpoints', 'src': 'o

In [3]:
env_ids = ["CartPole-v1", "HalfCheetah-v4"]
train_different_envs = []
for j, env_id in enumerate(env_ids):
    train_envs = make_batched_env(j, env_id, cfg, mode='train')
    train_different_envs.append(train_envs)
    print(f"{j+1}/{len(env_ids)}environment {env_id} is loaded...")


Train env 1: CartPole-v1 is loaded
1/2environment CartPole-v1 is loaded...


/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/kukjin/.conda/envs/darl2/lib/python3.10/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Train env 2: HalfCheetah-v4 is loaded
2/2environment HalfCheetah-v4 is loaded...


In [9]:
device = 'cpu'
envs_storages = TensorDict({}, batch_size=[128, 64])
for i, envs in enumerate(train_different_envs):
    env_id = env_ids[i]
    obs = torch.zeros((128, 64) \
        + envs.single_observation_space.shape).to(device)
    actions = torch.zeros((128, 64) \
        + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((128, 64)).to(device)
    rewards = torch.zeros((128, 64)).to(device)
    dones = torch.zeros((128, 64)).to(device)
    values = torch.zeros((128, 64)).to(device)
    storage = TensorDict({
                "obs": obs,
                "actions": actions,
                "logprobs": logprobs,
                "rewards": rewards,
                "dones": dones,
                "values": values
                }, batch_size=[128, 64])
    envs_storages[env_id] = storage
    

In [10]:
envs_storages

TensorDict(
    fields={
        CartPole-v1: TensorDict(
            fields={
                actions: Tensor(shape=torch.Size([128, 64]), device=cpu, dtype=torch.float32, is_shared=False),
                dones: Tensor(shape=torch.Size([128, 64]), device=cpu, dtype=torch.float32, is_shared=False),
                logprobs: Tensor(shape=torch.Size([128, 64]), device=cpu, dtype=torch.float32, is_shared=False),
                obs: Tensor(shape=torch.Size([128, 64, 4]), device=cpu, dtype=torch.float32, is_shared=False),
                rewards: Tensor(shape=torch.Size([128, 64]), device=cpu, dtype=torch.float32, is_shared=False),
                values: Tensor(shape=torch.Size([128, 64]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([128, 64]),
            device=None,
            is_shared=False),
        HalfCheetah-v4: TensorDict(
            fields={
                actions: Tensor(shape=torch.Size([128, 64, 6]), device=cpu, dtype=torch.float

In [11]:
reshaped_storages = envs_storages.reshape(-1)

In [12]:
reshaped_storages

TensorDict(
    fields={
        CartPole-v1: TensorDict(
            fields={
                actions: Tensor(shape=torch.Size([8192]), device=cpu, dtype=torch.float32, is_shared=False),
                dones: Tensor(shape=torch.Size([8192]), device=cpu, dtype=torch.float32, is_shared=False),
                logprobs: Tensor(shape=torch.Size([8192]), device=cpu, dtype=torch.float32, is_shared=False),
                obs: Tensor(shape=torch.Size([8192, 4]), device=cpu, dtype=torch.float32, is_shared=False),
                rewards: Tensor(shape=torch.Size([8192]), device=cpu, dtype=torch.float32, is_shared=False),
                values: Tensor(shape=torch.Size([8192]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([8192]),
            device=None,
            is_shared=False),
        HalfCheetah-v4: TensorDict(
            fields={
                actions: Tensor(shape=torch.Size([8192, 6]), device=cpu, dtype=torch.float32, is_shared=False),
  

In [33]:
class SumTensorDict(TensorDict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def __add__(self, other_dict):
        new_dict = SumTensorDict({}, batch_size=self._batch_size)    
        assert len(self._tensordict.keys()) == len(other_dict.keys())
        for key in self._tensordict.keys():
            if not (key in other_dict.keys()):
                raise KeyError
            new_dict[key] = self._tensordict[key] + other_dict[key]
        return new_dict
    
    def __sub__(self, other_dict):
        new_dict = SumTensorDict({}, batch_size=self._batch_size)    
        assert len(self._tensordict.keys()) == len(other_dict.keys())
        for key in self._tensordict.keys():
            if not (key in other_dict.keys()):
                raise KeyError
            if self._tensordict[key].shape != other_dict[key].shape:
                raise ValueError
            new_dict[key] = self._tensordict[key] - other_dict[key]
        return new_dict


In [37]:
cartpole_dict = SumTensorDict({'reward': 0.5*torch.ones([128,]),
                               'action': -1*torch.ones([128,]),
                               }, batch_size=128)
hafcheetah_dict = SumTensorDict({'reward': torch.ones([128,]),
                               'action': 2*torch.ones([128,]),
                                 }, batch_size=128)

In [38]:
print(cartpole_dict.keys())
print(hafcheetah_dict.keys())
print(len(cartpole_dict.keys()))
print(len(hafcheetah_dict.keys()))


_TensorDictKeysView(['reward', 'action'],
    include_nested=False,
    leaves_only=False)
_TensorDictKeysView(['reward', 'action'],
    include_nested=False,
    leaves_only=False)
2
2


In [11]:
# cartpole_dict = TensorDict({'reward': torch.zeros([128,])}, batch_size=128)
# hafcheetah_dict = TensorDict({'reward': torch.ones([128,])}, batch_size=128)


In [56]:
td = TensorDict({'reward': []}, batch_size=0)

In [57]:
td['reward']

tensor([])

In [39]:
sum_dict = cartpole_dict + hafcheetah_dict

In [45]:
cartpole_dict[:]

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([128]),
    device=None,
    is_shared=False)

In [44]:
sum_dict2 = cartpole_dict[:] + hafcheetah_dict[:]


TypeError: unsupported operand type(s) for +: 'TensorDict' and 'TensorDict'

In [43]:
sum_dict[0:3] # 딕셔너리 키의 값들을 인덱싱 가능 

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False),
        reward: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([3]),
    device=None,
    is_shared=False)

In [36]:
subtract_dict = cartpole_dict - hafcheetah_dict
subtract_dict['reward']

tensor([-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
        -0.5000, -0.5000, -0.5000, -0.50

In [31]:
sum_dict['reward']

tensor([1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 

In [32]:
sum_dict

SumTensorDict(
    fields={
        reward: Tensor(shape=torch.Size([128]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([128]),
    device=None,
    is_shared=False)

In [3]:
td = TensorDict({}, batch_size=128)

In [5]:
from torchrl.data.tensor_specs import CompositeSpec, BoundedTensorSpec, DiscreteTensorSpec

In [6]:
observation_vector_spec = BoundedTensorSpec(-torch.ones(17),torch.ones(17))

In [7]:
observation_vector_spec.dtype

torch.float32

In [8]:
action_vector_spec = BoundedTensorSpec(-torch.ones(6),torch.ones(6))
action_vector_spec

BoundedTensorSpec(
     shape=torch.Size([6]), space=ContinuousBox(minimum=Tensor(shape=torch.Size([6]), device=cpu, dtype=torch.float32, contiguous=True), maximum=Tensor(shape=torch.Size([6]), device=cpu, dtype=torch.float32, contiguous=True)), device=cpu, dtype=torch.float32, domain=continuous)

In [9]:
print(action_vector_spec.shape)
print(action_vector_spec.domain)


torch.Size([6])
continuous


In [10]:
action_vector_spec = DiscreteTensorSpec(2, shape=(1,))
action_vector_spec

DiscreteTensorSpec(
     shape=torch.Size([1]), space=DiscreteBox(n=2), device=cpu, dtype=torch.int64, domain=discrete)

In [1]:
from torchrl.envs.libs.gym import GymEnv
env = GymEnv("CartPole-v1", device="cpu")

In [2]:
env.env_name

'CartPole-v1'

In [3]:
env.action_spec

OneHotDiscreteTensorSpec(
     shape=torch.Size([2]), space=DiscreteBox(n=2), device=cpu, dtype=torch.int64, domain=discrete)

In [8]:
env.action_spec.rand()

tensor([0, 1])

In [4]:
obs = env.reset()

In [5]:
obs

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        is_continuous: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.float32, is_shared=False),
        output_dim: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)

In [9]:
obs['action'] = env.action_spec.rand()

In [10]:
out = env.step(obs)

In [11]:
out

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.int64, is_shared=False),
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        is_continuous: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.float32, is_shared=False),
        output_dim: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),

In [ ]:
obs

In [26]:
obs['env_index'] = 0

In [27]:
obs

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        env_index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False),
        observation: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)

In [31]:
env.action_spec.domain


'discrete'

In [29]:
obs['is_continuous'] = False
obs['output_dim'] = env.action_spec.shape



In [30]:
obs

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.bool, is_shared=False),
        env_index: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.int64, is_shared=False),
        is_continuous: Tensor(shape=torch.Size([]), device=cpu, dtype=torch.bool, is_shared=False),
        observation: Tensor(shape=torch.Size([4]), device=cpu, dtype=torch.float32, is_shared=False),
        output_dim: Tensor(shape=torch.Size([1]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([]),
    device=cpu,
    is_shared=False)

In [18]:
print(action_vector_spec.shape)
print(action_vector_spec.shape)


torch.Size([1])
